## Project: AI-BEMS 알고리즘 도출
* 2024.05.21. SQI에서 인천광역시 고등학교 17개소의 BEMS 데이터 공유

In [1]:
# 라이브러리를 수정할 때마다 autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os

In [2]:
file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/buildingID_devID_map.csv"
buildingID_devID_df = pd.read_csv(file_path)
print(buildingID_devID_df.head(3))

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/devID_tagCD_map.csv"
devID_tagCD_df = pd.read_csv(file_path)
print(devID_tagCD_df.head(3))

  buildingID  devID                    devName devType
0      B0001   1001     인천해양과학고등학교 인버터1_2.53kW     인버터
1      B0001   1002     인천해양과학고등학교 인버터2_2.53kW     인버터
2      B0001   1003  인천해양과학고등학교 신축인버터1_49.95kW     인버터
       devID_range devID_description  tagCD         tagName
0  devID 1001~1999   inverter 태양광인버터  10002           PV 전압
1  devID 1001~1999   inverter 태양광인버터  10003  계통 R상(RS선간) 전압
2  devID 1001~1999   inverter 태양광인버터  10004  계통 S상(ST선간) 전압


In [3]:
import numpy as np
import pandas as pd
import Utility as Util

file_path = '/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/data_colec_h_202410141057_B0008.csv'
df_buildID = pd.read_csv(file_path)

df_buildID['colec_dt'] = pd.to_datetime(df_buildID['colec_dt']).dt.floor('min')  # 분 이하는 제거
print(f"First timeStamp: {df_buildID['colec_dt'].iloc[0]= }, {df_buildID['colec_dt'].min()}")
print(f"Last timestamp: {df_buildID['colec_dt'].iloc[-1]= }, {df_buildID['colec_dt'].max()}")

dev_id_list = df_buildID['dev_id'].unique()
print(f"{len(dev_id_list)=}\n{dev_id_list=}")
Inverter = np.sum((dev_id_list >= 1000) & (dev_id_list < 2000))
ElecMeter = np.sum((dev_id_list >= 2000) & (dev_id_list < 3000))
GasMeter = np.sum((dev_id_list >= 3000) & (dev_id_list < 4000))
HeatMeter = np.sum((dev_id_list >= 4000) & (dev_id_list < 5000))
EnvSensor = np.sum((dev_id_list >= 5000) & (dev_id_list < 6000))
AirCon = np.sum((dev_id_list >= 6000) & (dev_id_list < 7000))
print(f'{Inverter=}, {ElecMeter=}, {GasMeter=}, {HeatMeter=}, {EnvSensor=}, {AirCon=}')
df_buildID.head(3)

First timeStamp: df_buildID['colec_dt'].iloc[0]= Timestamp('2023-01-01 00:00:00'), 2023-01-01 00:00:00
Last timestamp: df_buildID['colec_dt'].iloc[-1]= Timestamp('2024-10-14 10:30:00'), 2024-10-14 10:30:00
len(dev_id_list)=133
dev_id_list=array([2001, 2002, 2003, 2004, 3001, 3002, 3003, 3004, 3005, 4001, 4002,
       5001, 5002, 5003, 1001, 1002, 6001, 6002, 6003, 6004, 6005, 6006,
       6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015, 6016, 6017,
       6018, 6019, 6020, 6021, 6022, 6023, 6024, 6025, 6026, 6027, 6028,
       6029, 6030, 6031, 6032, 6033, 6034, 6035, 6036, 6037, 6038, 6039,
       6040, 6041, 6042, 6043, 6044, 6045, 6046, 6047, 6048, 6049, 6050,
       6051, 6052, 6053, 6054, 6055, 6056, 6057, 6058, 6059, 6060, 6061,
       6062, 6063, 6064, 6065, 6066, 6067, 6068, 6069, 6070, 6071, 6072,
       6073, 6074, 6075, 6076, 6077, 6078, 6079, 6080, 6081, 6082, 6083,
       6084, 6085, 6086, 6087, 6088, 6089, 6090, 6091, 6092, 6093, 6094,
       6095, 6096, 6097, 6098, 

,dev_id,tag_cd,colec_dt,colec_val,reg_dt,bldg_id
0,2001,202202,2023-01-01,1.00,2023-01-01 00:13:15.700,B0008
1,2001,30001,2023-01-01,49.00,2023-01-01 00:13:15.700,B0008
2,2001,40004,2023-01-01,413502.81,2023-01-01 00:13:15.700,B0008


### EDA

In [4]:
# 학사일정.... 용학초등학교
"""
2023년
03-02(목) ~ 07-21(금) : 입학식~여름방학식, semester1
07-22(토) ~ 08-21(월) : 여름방학시작~종료, summerVacation
08-22(화) ~ 12-29(금) : 여름방학개학식~겨울방학식, semester2
12-30(토) ~ 01-23(화) : 겨울방학시작~종료, winterVacation
01-24(수) ~ 01-31(수) : 겨울방학개학식~졸업식, winterSession 
02-01(목) ~ 03-03(일) : 졸업방학시작~종료, graduationVacation

03-04(월) ~ 07-24(수) : 입학식~여름방학식, semester1
"""

# 일주일 고찰
# 2024년 1월 21일(일)~27일(토)
start_date, end_date = pd.to_datetime('2024-01-21 00:00:00'), pd.to_datetime('2024-01-28 00:00:00')
# 2024년 3월 17일(일)~23일(토)  
start_date, end_date = pd.to_datetime('2024-03-17 00:00:00'), pd.to_datetime('2024-03-24 00:00:00')  
# 2024년 7월 16일(일)~24일(토)
start_date, end_date = pd.to_datetime('2023-07-16 00:00:00'), pd.to_datetime('2023-07-25 00:00:00')  


# 이틀 고찰
# winterVacation-난방, 2024년 1월 22일(월)~23일(화)
start_date, end_date = pd.to_datetime('2024-01-22 00:00:00'), pd.to_datetime('2024-01-24 00:00:00') 
# semester1-난방, 2024년 3월 18일(월)~19일(화) 
start_date, end_date = pd.to_datetime('2024-03-18 00:00:00'), pd.to_datetime('2024-03-20 00:00:00')  
# semester1-냉방, 2023년 7월 17일(월)~18일(화)
start_date, end_date = pd.to_datetime('2023-07-17 00:00:00'), pd.to_datetime('2023-07-19 00:00:00')

# 관찰 구간
start_date, end_date = pd.to_datetime('2023-07-20 00:00:00'), pd.to_datetime('2023-07-26 00:00:00')
start_date, end_date = pd.to_datetime('2024-01-22 00:00:00'), pd.to_datetime('2024-01-24 00:00:00') 
start_date, end_date = pd.to_datetime('2024-01-22 00:00:00'), pd.to_datetime('2024-01-26 00:00:00') 

df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

### Device Analysis

#### 학교 Main

In [6]:
start_date, end_date = pd.to_datetime('2023-01-01 00:00:00'), pd.to_datetime('2024-10-14 10:30:00')
df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

#devID==2001 "학교MAIN", tagCD==30001 "현재출력", peak_prediction()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == 'B0008') & (buildingID_devID_df['devID'].isin([2001]))])

devID, device_name = 2001, '학교MAIN'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량_차', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량', 40007: '역송전력량', 202202: 'validData'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'
#tags[40007] = calc_tagCD_diff(tags[40007])  # '역송전력량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

# for training_lightGBM_Base.ipynb
df_raw = pd.DataFrame({'value': tag_data[0]['colec_val'].values}, index=tag_data[0]['colec_dt']).rename_axis('timestamp')
Util.plot_data(df_raw, plotType='simple', title=device_name, W=20, H=10)
import pickle
df_raw.to_pickle('df_raw.pickle')

    buildingID  devID  devName devType
905      B0008   2001  학교 MAIN   전기계측기
tagCD list:  [202202  30001  40004  50001  40007]


AttributeError: module 'Utility' has no attribute 'plot_devID_tagCD'

### Device Analysis(old)

In [11]:
devID, device_name = 1001, '인버터1'
Util.print_tagCD(df, devID)
tag_dict = {10002: 'PV전압', 20002: 'PV전류', 30001: '현재출력', 30002: 'PV출력', 60001: '주파수', 70006: '온도', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)


tagCD list:  [ 202202   10002   20002   30002   10003   10004   10005   20003   20004
   20005   30001   40001   60001     703 9003005 9003007 9003008 9003014
 9003020 9003023 9003030 9003034 9003038 9003040 9003041 9003042 9003045
 9003047 9003048 9003049 9003081 9003106]


AttributeError: module 'Utility' has no attribute 'plot_devID_tagCD'

In [8]:
devID, device_name = 2001, '학교MAIN'
print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량_차', 50001: '역률', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'
#tags[40007] = calc_tagCD_diff(tags[40007])  # '역송전력량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

NameError: name 'print_tagCD' is not defined

In [ ]:
devID, device_name = 2002, 'EHP-1'
print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

In [ ]:
# "EHP-1", "EHP-2", "EHP-4" 비교
device_dict = {
    2002: "EHP-1",
    2003: "EHP-2",
    2004: "EHP-4",
}

#tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'selfcheck'}
tag_dict = {30001: '현재 출력'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="all EHPs", createFig=False, W=None, H=600)

In [5]:
devID, device_name = 3002, '식당급탕보일러'
print_tagCD(df, devID)
tag_dict = {70002: '보정적산량_차', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[70002] = calc_tagCD_diff(tags[70002])  # '보정적산량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)


NameError: name 'print_tagCD' is not defined

In [ ]:
devID, device_name = 4002, '본관급탕보일러'
print_tagCD(df, devID)
tag_dict = {90001: '공급온도', 90002: '환수온도', 90003: '순시유량', 90004: '누적유량_차', 90005: '순시열량', 90006: '누적열량_차', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[90004] = calc_tagCD_diff(tags[90004])
tags[90006] = calc_tagCD_diff(tags[90006])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)


In [ ]:
devID, device_name = 5003, '외기온도'
print_tagCD(df, devID)
tag_dict = {80002: '온도', 80003: '습도', 80004: 'CO2', 80005: '미세먼지', 80006: '초미세먼지', 80007: '조도', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

In [ ]:
devID==1001 1002 "인버터1" "인버터2", tagCD==30001 "현재출력", anomaly_detection()
devID==2001 "학교MAIN", tagCD==30001 "현재출력"
devID==2002 2003 2004 "EHP-1" "EHP-2" "EHP-4", tagCD==30001 "현재 출력", peak_prediction(), anomaly_detection()
devID==3001~3005 "식당급탕보일러", tagCD==70002 '보정적산량_차', anomaly_detection(), vacation_detection()~50% 선
devID==4001 4002, "본관급탕보일러(열량계)", tagCD==90006 "누적열량_차", anomaly_detection()
devID==5001 5002 5003 "외기온도", tagCD==80002, anomaly_detection()       
devID==6001~6117 "에어콘", tagCD==100003, "현재온도", anomaly_detection()
devID==6001~6117 "에어콘", tagCD==, control_estimation()
devID==6001~6117 "에어콘", tagCD==, control_prediction()     
                100001	운전선택탭  off: 0, on: 1
                100002	냉난방모드	cool: 0, heat: 1, dry: 2, fan: 3, auto: 4, normal: 5, env: 6, sleep: 6
                100003	현재온도	현재온도
                100004	희망온도	냉방 18.0~30.0, 난방 16.0~30.0
                100005	풍량        low: 0, med: 1, mid: 1, high: 2, auto: 3
                100006	리모컨허용  level: 2, true: 1, false: 0
                100007	에러	    indoor: 0, outdoor: 1, caur: 2, trans: 3

In [ ]:
devID, device_name = 6001, '4-7반_본405'
print_tagCD(df, devID)
tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도', 100005: '풍량', 100006: '리모컨허용', 100007: '에러', 202202: 'selfcheck'}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, createFig=True, W=None, H=600)

In [ ]:
device_dict = {
    6007: "1-6반_본108",
    6008: "1-5반_본107",
    6009: "1-4반_본106",
    6010: "1-3반_본105",
    6028: "1-7반_본210",
    6029: "1-8반_본209",
}
print_tagCD(df, devID)

tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="1학년", createFig=False, W=None, H=600)

In [ ]:
print_tagCD(df, devID)
device_dict = {
    6044: "2-2반_후307-1",
    6045: "2-5반_후304",
    6046: "2-4반_후303",
    6052: "2-7반_후306",
    6053: "2-6반_후305",
    6055: "2-1반_본304",
    6057: "2-1반_본304",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="2학년", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6001: "4-7반_본405",
    6082: "4-2반_후407",
    6083: "4-1반_후407-1",
    6084: "4-1반_후407-1",
    6093: "4-3반_본401",
    6094: "4-4반_본402",
    6095: "4-5반_본403",
    6111: "4-6반_본404",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="4학년", createFig=False, W=None, H=600)
print_tagCD(df, devID)

In [ ]:
device_dict = {
    6002: "유치원복도1",
    6004: "누리반_후107",
    6005: "한울반_후106",
    6006: "가람반_후105",
    6116: "아라반_후103",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="유치반", createFig=False, W=None, H=600)
print_tagCD(df, devID)


In [ ]:
device_dict = {
    6041: "돌봄협의실_후302",
    6042: "돌봄자료실_후307",
    6043: "방과후코디실_후307-1",
    6077: "연계형돌봄교실_후311",
    6107: "돌봄준비실_후301-1",
    6108: "돌봄교실1_후301",
    6109: "돌봄교실1_후301",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="돌봄교실", createFig=False, W=None, H=600)
print_tagCD(df, devID)


In [ ]:
device_dict = {
    6003: "유치원교사실_후108",
    6023: "3학년연구실_후203",
    6096: "5학년교사연구실_본410",
    6098: "2학년교사연구실_본311",
    6101: "6학년교사실_후508",
    6102: "1학년교사연구실_본109",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="교사연구실", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6079: "학부모실_후310",
    6081: "4협의실_후402",
    6089: "휴게실_급109-1",
    6092: "위클래스_후201",
    6097: "남교사 휴게실_후503",
    6099: "여교사 휴게실_후502",
    6100: "위클래스_본211",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="휴게실 등", createFig=False, W=None, H=600)

In [ ]:
# 일주일 고찰
# 2024년 1월 21일(일)~27일(토)
start_date, end_date = pd.to_datetime('2024-01-21 00:00:00'), pd.to_datetime('2024-01-28 00:00:00')
# 2024년 3월 17일(일)~23일(토)  
start_date, end_date = pd.to_datetime('2024-03-17 00:00:00'), pd.to_datetime('2024-03-24 00:00:00')  
# 2024년 7월 16일(일)~22일(토)
start_date, end_date = pd.to_datetime('2023-07-16 00:00:00'), pd.to_datetime('2023-07-23 00:00:00')  

df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

In [ ]:
device_dict = {
    6011: "체육관후면좌_후210",
    6012: "체육관전면좌_후210",
    6047: "체육관후면우_후210",
    6048: "체육관전면우_후210",
}

tag_dict = {100003: '현재온도', 100004: '희망온도'}
#tag_dict = {100003: '현재온도'}

"""device_dict = {
    6012: "체육관전면좌_후210",
}
tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도', 100005: '풍량', 100006: '리모컨허용', 100007: '에러', 202202: 'selfcheck'}"""

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="체육관", createFig=False, W=None, H=600)

In [ ]:
# 이틀 고찰
# winterVacation-난방, 2024년 1월 22일(월)~23일(화)
start_date, end_date = pd.to_datetime('2024-01-22 00:00:00'), pd.to_datetime('2024-01-24 00:00:00') 
# semester1-난방, 2024년 3월 18일(월)~19일(화) 
start_date, end_date = pd.to_datetime('2024-03-18 00:00:00'), pd.to_datetime('2024-03-20 00:00:00')  
# semester1-냉방, 2023년 7월 17일(월)~18일(화)
start_date, end_date = pd.to_datetime('2023-07-17 00:00:00'), pd.to_datetime('2023-07-19 00:00:00')

df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

In [ ]:
device_dict = {
    6013: "음악실1_본501-2",
    6014: "음악실1_본501-2",
    6016: "음악준비실_본501-4",
    6018: "개인연습실2_본501-5",
    6020: "개인연습실1_본501-6",
    6022: "무용실_본501",
    6024: "무용실_본501",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="예능실", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6025: "컴준비실_후208",
    6026: "컴퓨터실_후208-1",
    6027: "컴퓨터실_후208-1",
    6049: "시청각실_후509",
    6050: "시청각실_후509",
    6051: "시청각실_후509",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="전산실", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6033: "행정실2_본203",
    6034: "행정실1_본203",
    6035: "탕비실_본204-2",
    6036: "교무실2_본202",
    6037: "교무실1_본202",
    6038: "교장실_본201",
    6039: "학교운영위원회실_본212",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="교무실", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6054: "식당전4_급101",
    6056: "식당전3_급101",
    6058: "식당전2_급101",
    6060: "식당전1_급101",
    6062: "식당중3_급101",
    6085: "세척실_급103",
    6086: "조리실2_급105",
    6087: "조리실1_급105",
    6088: "영양사실_급106",
    6090: "전처리실_급109",
    6091: "식품창고_급107",
    6105: "식당중2_급101",
    6106: "식당중1_급101",
    6117: "식당후면2_급101"
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="식당시설", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6067: "2층도서실_후202",
    6069: "2층도서실_후202",
    6072: "2층도서실_후202",
    6074: "2층도서실_후202",
    6110: "3층도서실덕트",
    6112: "3층도서실",
    6113: "3층도서실",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="도서실", createFig=False, W=None, H=600)

In [ ]:
device_dict = {
    6059: "전산실_본303",
    6061: "인쇄실_본302",
    6063: "스튜디오_본301-1",
    6064: "방송실_본301",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="방송실", createFig=False, W=None, H=600)